In [19]:
"use strict";
var http = require('http');
var https = require("https");
var querystring = require('querystring');
var fetch = require('node-fetch');
var xml2js = require('xml2js');
var url = require('url');
var fs = require('fs');
var os = require('os');
var path = require('path');

'use strict'

In [2]:
var XMLParser = class {
    constructor(xml) {
        this._result = this.parse(xml);
    }
    parse(xml) {
        if (typeof xml == "object") {
            return this.parse_object(xml);
        } else {
            throw "Don't know how to parse " + typeof xml;
        }
    }
    parse_object(xml) {
        let ret = {};
        for(let [key, value] of Object.entries(xml)){
            if (key == "Cluster") {
                let ob = {};
                let obname = "Cluster";
                for(let [key2, value2] of Object.entries(value[0])) {
                    if (key2 == "Name") {
                        obname = value2[0] || obname;
                    } else if (key2 == "NumElts") {
                        continue
                    } else {
                        // value2 is array of children.............
                        for (let v3 of value2){
                            let name = v3.Name[0];
                            let val = v3.Val[0];
                            ob[name] = val;
                        }
                    }
                }
                ret[obname] = ob;
            } else {
                throw "???";
            }

        }
        return ret;
    }
}

var ParseXML = function(xml) {
    var parser = new XMLParser(xml.Reply.Message[0]);
    return parser._result;
}


var HelloApp = class {
    constructor (ipaddr, https=true) {
        
        var u = url.parse(ipaddr);
        if (u.protocol == "http:") {
            https = false;
        } else if (u.protocol == "https:") {
            https = true;
        }
        
        this._url = `${ https ? "https" : "http" }://${u.hostname}:${u.port || 80}/webservice`;
        
        this._url_interface = `${this._url}/interface`;
        this._url_report = `${this._url}/getreport`;
        this._url_getfile = `${this._url}/getfile`;
    }

    _innercall(url, json) {
        var rsp = fetch(url);
        var cb;
        if (json) {
            return rsp.then(r => r.json());
        } else {
            return rsp
                .then(r => r.text())
                .then(text => xml2js.parseStringPromise(text));
        }
    }
    _docall(url, args, json) {
        if (args != undefined){
            url = url + "?" + querystring.encode(args);
        }
        return this._innercall(url, json);
    }
    call(server_call, args, json) {
        if (typeof server_call == "object") {
            if (typeof args != "undefined") {
                throw "Args must be null if server_call is object";
            }
            args = server_call;
        } else if (typeof server_call == "string") {
            args = args || {};
            args.call = server_call;
        }
        return this._docall(this._url_interface, args, json == undefined ? false : json)
    }
    
    getMainValues() {
        return this.call("getMainValues", {json:true}, true);
    }
    getStatic() {
        return this.call("getStatic", {json:true}, true);
    }
    getDORAValues () {
        var xml = this.call("getDORAValues", {}, false);
        return xml.then(ParseXML).then(r => r.Cluster);
    }
}

In [32]:
var getExt = function (fname) {
    return fname.slice((Math.max(0, fname.lastIndexOf(".")) || Infinity) + 1);
}

var _extensions = {};
_extensions['json'] = 'Application/json';
_extensions['css'] = 'text/css';
_extensions['png'] = 'image/png';
_extensions['js'] = 'application/x-javascript';
_extensions['gif'] = 'image/gif';
_extensions['ico'] = 'image/icon';
_extensions['html'] = 'text/html';
_extensions['woff'] = 'font/woff';
_extensions['ttf'] = 'application/octet-stream';

var getContentType = function(file) {
    var ext = getExt(file);
    return _extensions[ext] || "text/plain";
}

var g_headers = null;
var g_request = null;
var g_this = null;

var cookie_key = function(request){
    let ip = request.connection.remoteAddress;
    let port = request.connection.remotePort;
    return ip;
}

var ProxyServer2 = class {
    
    constructor(port, fwd_url, dirloc) {
        
        this.requestHandler = this.requestHandler.bind(this);
        this.doProxy = this.doProxy.bind(this);
        this.handleCall = this.handleCall.bind(this);
        this.makeProxyRequest = this.makeProxyRequest.bind(this);
        this.getDefaultResponseForCall = this.getDefaultResponseForCall.bind(this);
        this.handleCookie = this.handleCookie.bind(this);
        
        this._port = port;

        var u = url.parse(fwd_url);
        
        this._directory = dirloc;
        this._proto = u.protocol;
        this._host = u.hostname;
        this._fwd_port = u.port;
        
        this._server = http.createServer(this.requestHandler);
        this._handlers = {};
        this._agent = new https.Agent({
            keepAlive: true,
            keepAliveMsecs: 100000,
            maxSockets: 1
        });
        
        this._PBSCookies = {};
        
    }
    
    run() {
        this._server.listen(this._port);
    }
    stop() {
        this._agent.destroy();
        this._server.close();
    }
    
    register(call, handler) {
        if (typeof handler != "function") {
            throw "handler must be function, got " + typeof handler;
        }
        this._handlers[call.toLowerCase()] = handler;
    }
    
    unregister(call) {
        this._handlers[call.toLowerCase()] = undefined;
    }
    
    async handleCall(request, response, args){
        var callback = this._handlers[args.call.toLowerCase()];
        if (callback != undefined){
            try {
                
                var interceptEventArg = {
                    query: args,
                    request: request,
                    response: response,
                    server: this,
                    getDefaultResponse: () => this.getDefaultResponseForCall(request),
                }
                
                var rsp = await callback(interceptEventArg);
                if (rsp != undefined){
                    response.statusCode = 200;
                    response.setHeader('Content-Type', rsp.content_type || "application/json");
                    response.setHeader('Content-Length', Buffer.byteLength(rsp.body));
                    response.end(rsp.body);
                }
            } catch (e) {
                var msg = e.toString();
                response.statusCode = 500;
                response.setHeader('Content-Type', "text/plain");
                response.setHeader('Content-Length', Buffer.byteLength(msg));
                response.end(msg)
            }
            return true;
        }
        return false;
    }
    
    makeProxyRequest(request) {
        var opts = {
            headers: request.headers,
            agent: this._agent,
            method: request.method,
            host: this._host,
            port: this._fwd_port,
            protocol: this._proto,
            path: request.url,
            rejectUnauthorized: false,
        };
        let key = cookie_key(request);
        let cookie = this._PBSCookies[key];
        if (cookie != undefined) {
            opts.headers['cookie'] = cookie;
        }
        return https.request(opts);
    }
    
    async getDefaultResponseForCall(request) {
        return new Promise((resolve, reject) => {
            var req = this.makeProxyRequest(request);
            var data = [];
            req.addListener("response", response => {
                response.on("data", chunk => {
                    data.push(chunk);
                });
                response.on("end", () => resolve(data.join("")));
                response.on("error", err => reject(err));
            });
            req.end();
        });
    }
    
    handleCookie(request, proxy_response, response){
        
        let key = cookie_key(request);
        
        if (proxy_response.headers['set-cookie'] != undefined){
            let parts = proxy_response.headers['set-cookie'][0].split(";");
            this._PBSCookies[key] = parts[0];  // "key=value"
        }
    }
    
    doProxy(request, response) {
        /* Executes a transparent proxy request */
        var self = this;
        var proxy_request = this.makeProxyRequest(request);
        g_request = proxy_request;
        proxy_request.addListener('response', function (proxy_response) {
            proxy_response.addListener('data', chunk => {
                response.write(chunk, 'binary');
            });
            proxy_response.addListener('end', () => response.end());
            self.handleCookie(request, proxy_response, response);
            
            response.writeHead(proxy_response.statusCode, undefined, proxy_response.headers);
        });
        
        request.addListener('data', chunk => proxy_request.write(chunk, 'binary')); 
        request.addListener('end', () => proxy_request.end());
    }
    
    async requestHandler (request, response) {
        
        /* Main request handler. 
         *
         * Intercepts & all requests. Request handlers registered with
         * the server will be used to return an alternate response.
         * If no handler exists, fallthrough to the proxy method. 
         * 
         * Server Calls -> check handler registry 
         * filename -> check local hosting directory
         *
         * Alt server calls (getreport, getfile) not handled!
         */
        
        var u = url.parse(request.url);
        var args = querystring.parse(u.query);
        
        // urls are fun! /foo//bar/ -> foo/bar
        var path = u.pathname.split("/").filter(o => o).join("/");
        
        if (path == "webservice/interface"){
            var handled = await this.handleCall(request, response, args);
            if (handled) {
                return; // handled
            }
        } else if (path == "webservice/getreport") {
            // pass
        } else if (path == "webservice/getfile") {
            // pass
        } else {
            // file
            if (this._directory) {
                var full = this._directory + "/" + u.pathname;
                var sanitized = full.replace(/\\/g, "/")            // \foo\..\.\baz -> /foo/.././bar
                                    .replace(/\/\.+(?=\/)/g, "/")   // /foo/.././bar -> foo///bar
                                    .replace(/\/{2,}/g, "/");       // /foo///bar -> foo/bar
                var mime = getContentType(sanitized);
                
                fs.readFile(sanitized, null, (err, bytes) => {
                    var length = Buffer.byteLength(bytes);
                    response.statusCode = 200;
                    response.setHeader('Content-Type', mime);
                    response.setHeader('Content-Length', length);
                    response.end(bytes);
                });
                return;
            }
        }
        
        // fallthrough
        this.doProxy(request, response);
    }
}

'application/octet-stream'

In [33]:
var showYield = function(gs) {
    var msg = `enableLevel: ${gs.level.enableLevel}, pH: ${gs.ph.inputs}, DO: ${gs.do.inputs}, Temp: ${gs.temperature.inputs}`;
    console.log(msg);
}

var getStaticLoop = function(done_cb) {
    var state = {
        coro: undefined,
        done: false
    }
    var cycle = function*() {
        // init
        
        var gs = yield;
        
        // level = 0
        gs['level']['enableLevel'] = 0;
        gs['ph']['inputs'] = 0;
        gs['do']['inputs'] = 0;
        gs['temperature']['inputs'] = 0;
        gs = yield gs;
        
        // ph, do, temp all combo
        for (var ph = 0; ph < 3; ph++) {
            for (var dox = 0; dox < 3; dox++){
                for (var temp = 0; temp < 3; temp++) {
                    gs['level']['enableLevel'] = 1;
                    gs['ph']['inputs'] = ph;
                    gs['do']['inputs'] = dox;
                    gs['temperature']['inputs'] = temp;
                    gs = yield gs;
                }
            }
        }
        state.coro = undefined;
        state.done = true;
        yield gs;
    }
    var handler = function(gs) {
        if (state.coro == undefined) {
            state.done = false;
            console.log("Creating new getStatic Coroutine")
            state.coro = cycle();
            state.coro.next();  // advance to 1st yield
        }
        var res;
        res = state.coro.next(gs).value; // feed
        showYield(res);
        if (state.done && done_cb && typeof done_cb != undefined){
            done_cb();
        }
        return res;
    }
    return handler;
}

In [34]:
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var test = async function() {
    var state = {stop: false};
    var app = new HelloApp('localhost:12348', false);
    var gs = await app.getStatic();
    handler = getStaticLoop(() => state.stop = true);
    while (!state.stop) {
        handler(gs);
        await sleep(100);
    }
    
    
}
var test2 = async function() {
    var app = new HelloApp('localhost:12348', false);
    var gs = await app.getStatic();
    var state = {run:true};
    handler = getStaticLoop(()=> state.run=false);
    for (var i = 0; i < 100; ++i){
        handler(gs);
    }
}

In [45]:
// helpers for managing copies of local server response files

var _home_dir = path.join(os.homedir(), "desktop\\CodeStuff");
var get_home_dir = function() {
    return _home_dir;
}

var set_home_dir = function(dir) {
    _home_dir = dir;
}

var make_home_dir = function() {
    var home = get_home_dir();
    try {
        fs.mkdirSync(home, {recursive:true});
    }
    catch (e){
        if (e.code != 'EEXIST')
            throw e;
    }
}

var makeFileForEvent = async function(ev, path) {
    var data = await ev.getDefaultResponse();
    fs.writeFileSync(path, data);
    return data;
}

var eventResponse = function(content_type, body) {
    return {
        content_type: content_type,
        body: body
    };
}

// each hook is an async function that gets an event object with the following keys:
//     - query: object of query string parameters {key: value}
//     - request: incoming request object
//     - response: outgoing response object
//     - server: the ProxyServer2 object 
//     - getDefaultResponse: function that forwards the request to remote server and returns the response. 

// hooks must return EITHER `undefined` (to trigger the default response)
// OR return an object with the following keys:
//     - content_type: content type of the respones body, e.g. 'application/json'
//     - body: response body as a string, e.g. JSON string for getMainValues


// example response for getStatic 

var hookGetStatic = function() {
    var gsloop = getStaticLoop();
    p.register("getstatic", async (ev) => {
        var gs = await ev.getDefaultResponse();
        gs = JSON.parse(gs);
        var result = gsloop(gs);
        return eventResponse("application/json", JSON.stringify(result));
    });
}

// generic hook for returning file responses

var useFileInstead = async function(ev) {
    var home = get_home_dir();
    var filename = ev.query.call;
    var ext = ev.query.json ? ".json" : ".xml"; // likely guess
    var filepath = path.join(home, filename + ext);
    var data = undefined;
    if (!fs.existsSync(filepath))
        data = makeFileForEvent(ev, filepath);
    else
        data = fs.readFileSync(filepath);
    var ct = ev.query.json ? "application/json" : "application/xml";
    return eventResponse(ct, data);
}

In [ ]:
try {
    p.stop();
} catch (e) {
    // pass
}
var proxy_url = "https://192.168.5.81";
var dirloc = "C:\\Users\\Nathan\\documents\\personal\\test\\uidev_311_dualsens";
var p = new ProxyServer2(12350, proxy_url, dirloc);
p.run();

In [49]:
// set up the home directory for server call response files
set_home_dir(path.join(os.homedir(), "\\desktop\\codestuff"));
make_home_dir();

// register a call for a file-based response. 
// If the file does not already exist, it will be created automatically
// on the next incoming request for this server call. 
p.register('getmainvalues', useFileInstead);

In [50]:
// remove a server call hook
p.unregister('getmainvalues');

In [51]:
// shutdown the server
p.stop()